In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import duckdb
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import BitVectToText
from rdkit.Chem import Descriptors, rdFingerprintGenerator

In [2]:
data_dir = "/home/pervinco/Datasets/leash-bio"
save_dir = f"{data_dir}/split_sets"

train_csv = f"{data_dir}/train.csv"
test_csv = f"{data_dir}/test.csv"

train_parquet = f"{data_dir}/train.parquet"
test_parquet = f'{data_dir}/test.parquet'

# 1.데이터셋 분석

In [ ]:
con = duckdb.connect()

## binds=0인 데이터만 load
count_binds_0 = con.query(f"""SELECT COUNT(*) 
                              FROM parquet_scan('{train_parquet}') 
                              WHERE binds = 0""").fetchone()[0]
print(f"Total binds=0 : {count_binds_0}")

## binds=1인 데이터만 load
count_binds_1 = con.query(f"""SELECT COUNT(*) 
                              FROM parquet_scan('{train_parquet}') 
                              WHERE binds = 1""").fetchone()[0]
print(f"Total binds=1 : {count_binds_1}")

## 전체 데이터 수
total_count = count_binds_0 + count_binds_1
print(f"Total data : {total_count}")

con.close()

In [ ]:
## 컬럼별 고유한 값, 갯수 파악

columns = [
    'buildingblock1_smiles', 
    'buildingblock2_smiles', 
    'buildingblock3_smiles', 
    'molecule_smiles', 
    'protein_name'
]

con = duckdb.connect()
for column in columns:
    query = f"SELECT {column}, COUNT(*) as count FROM parquet_scan('{train_parquet}') GROUP BY {column}"
    df = con.query(query).df()

    df.to_csv(f"{data_dir}/info/{column}_info.csv", index=False)

con.close()

In [ ]:
## 컬럼별 중복 데이터 확인

con = duckdb.connect()
bb1_query = f"SELECT DISTINCT buildingblock1_smiles FROM parquet_scan('{train_parquet}')"
bb2_query = f"SELECT DISTINCT buildingblock2_smiles FROM parquet_scan('{train_parquet}')"
bb3_query = f"SELECT DISTINCT buildingblock3_smiles FROM parquet_scan('{train_parquet}')"

bb1_set = set(con.query(bb1_query).df()['buildingblock1_smiles'])
bb2_set = set(con.query(bb2_query).df()['buildingblock2_smiles'])
bb3_set = set(con.query(bb3_query).df()['buildingblock3_smiles'])

bb1_bb2_intersection = bb1_set.intersection(bb2_set)
bb1_bb3_intersection = bb1_set.intersection(bb3_set)
bb2_bb3_intersection = bb2_set.intersection(bb3_set)

print(f"Building block 1 & 2 중복 : {'있음' if bb1_bb2_intersection else '없음'}")
print(f"Building block 1 & 3 중복 : {'있음' if bb1_bb3_intersection else '없음'}")
print(f"Building block 2 & 3 중복 : {'있음' if bb2_bb3_intersection else '없음'}")

print(f"Building block 1과 2 사이의 중복된 값: {bb1_bb2_intersection}")
print(f"Building block 1과 3 사이의 중복된 값: {bb1_bb3_intersection}")
print(f"Building block 2와 3 사이의 중복된 값: {bb2_bb3_intersection}")

con.close()

# 2.RDKit을 활용한 분석

In [3]:
def fetch_data(binds, offset, chunk_size):
    query = f"""
    SELECT *
    FROM parquet_scan('{train_parquet}')
    WHERE binds = {binds}
    ORDER BY random()
    LIMIT {chunk_size} OFFSET {offset}
    """
    return con.query(query).df()

In [4]:
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def compute_fingerprint(mol):
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    return BitVectToText(fp)  # Convert to BitString for storage

In [5]:
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}
    descriptors = Descriptors.CalcMolDescriptors(mol)
    return descriptors

In [6]:
"""
molecule_smiles는 building block들로 조합된 약물 분자.
모델 학습을 위해서는 인코딩을 적용해 컴퓨터가 이해할 수 있는 형태로 변환해야함.
"""

OFFSET = 0
CHUNK_SIZE = 10000
train_parquet = f'{data_dir}/train.parquet'
con = duckdb.connect()

output_dir = f"{data_dir}/preprocessed"
output_file = f"{output_dir}/train.parquet"
os.makedirs(output_dir, exist_ok=True)


first_chunk = True
while True:
    chunk = con.execute(f"""
    SELECT *
    FROM parquet_scan('{train_parquet}')
    LIMIT {CHUNK_SIZE} OFFSET {OFFSET}
    """).fetch_df()

    if chunk.empty:
        break

    ## 1.SMILES 문자열을 RDKit 객체로 변환.
    chunk['molecule'] = chunk['molecule_smiles'].apply(Chem.MolFromSmiles)

    ## 2.FingerPrint(해시 기반 이진 벡터) 생성.
    chunk['fingerprints'] = chunk['molecule'].apply(compute_fingerprint)

    ## 3.분자식으로부터 추가적인 특징들을 계산.
    descriptors_list = chunk['molecule_smiles'].apply(calculate_descriptors).tolist()
    descriptor_df = pd.DataFrame(descriptors_list)
    excluded_descriptors = descriptor_df.columns[descriptor_df.isna().any()].tolist()
    descriptor_df.drop(columns=excluded_descriptors, inplace=True)
    used_descriptor = descriptor_df.columns.tolist()

    if first_chunk:
        print(f"제외된 descriptors: {excluded_descriptors}")
        print(f"사용된 descriptors: {used_descriptor}")

    ## molecule 컬럼을 데이터프레임에서 제외
    chunk.drop(columns=['molecule'], inplace=True)

    ## 원래 데이터와 합치기.
    chunk = pd.concat([chunk, descriptor_df], axis=1)

    ## Parquet 파일로 저장.
    table = pa.Table.from_pandas(chunk)

    if first_chunk:
        writer = pq.ParquetWriter(output_file, table.schema)
        first_chunk = False
    
    writer.write_table(table)
    
    print(f"Processed offset: {OFFSET} saved to {output_file}")
    OFFSET += CHUNK_SIZE

    # break

writer.close()
con.close()

제외된 descriptors: ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']
사용된 descriptors: ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA

In [ ]:
df = pd.read_parquet(output_file, engine='pyarrow')
df.head()

In [ ]:
print(df.columns)
print(df.shape)